In [43]:
#!conda install python-graphviz

In [44]:
#!conda install python-graphviz

In [45]:
#!conda install python-graphviz

In [46]:
#!conda install python-graphviz

In [47]:
#!conda install python-graphviz

In [48]:
#!pip install graphviz

In [49]:
import os
import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker import get_execution_role
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
sagemaker_session = sagemaker.Session()
role = get_execution_role()
my_region = sagemaker_session.boto_session.region_name
import statsmodels.formula.api as smf

In [50]:
my_region

'ap-south-1'

In [51]:
# Read dataset through S3 bucket
training_data_uri = 'Latest_CV_DATA_HEADER.csv'.format(my_region)

In [52]:
training_data_uri

'Latest_CV_DATA_HEADER.csv'

In [53]:
#!pip install pandas-profiling
#!pip install scipy

In [54]:
import pandas as pd
import numpy as np
#import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.float_format', lambda x: '%0.3f' % x)
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)
import warnings
warnings.filterwarnings('ignore')

In [55]:
df = pd.read_csv('/root/PRICE-X/V1/Latest_CV_DATA_HEADER.csv',encoding='latin-1')
df_inf = pd.read_csv('/root/PRICE-X/V1/INFLAalpha.csv')

In [56]:
#cwd = os.getcwd()


In [57]:
#cwd

In [58]:
#os.chdir('/root/PRICE-X/PriceX')

In [59]:
#os.getcwd()

In [60]:
df.columns


Index(['UNIQUESERIALNO', 'SOLD_DATE', 'MAKE YEAR', 'Make', 'Model', 'Variant',
       'FUEL TYPE_CLEANED', 'mixmmvfuelcleaned', 'BodyType', 'SUBBODYTYPE',
       'WHEELS', 'GVW_HP_SEATING_CAPACITY_CC', 'SOLD AMOUNT', 'STATE_MAPPED',
       'VEHICLE NO', 'METER READING', 'UCR', 'SHROTCOND_MAPPED', 'INSDT',
       'TAX', 'PERMIT_MAPPED', 'SELLERCORPORATENAME', 'SELLERID', 'SELLERNAME',
       'KEYREFNUMBER', 'INV_H_RC_STATUS_CLEANED_MAPPED', 'VEHICLECONDITION',
       'SELLER SEGMENT'],
      dtype='object')

In [61]:
df_inf.columns

Index(['Year ', 'Price ', 'Inflation rate'], dtype='object')

In [62]:
df_inf.rename(columns={'Year ':'Year_sell'},inplace=True)

In [63]:
#df_inf

In [64]:
data=df.copy()

In [65]:
data.shape

(401773, 28)

In [66]:
def uppper(df,col):
    df[col]=df[col].str.upper()
    df[col]=df[col].str.replace(' ','')
    return df[col]

In [67]:
def encoding(data,col,col_1,model):
    print(model.head())
    temp = pd.read_csv(data+'.csv',encoding='CP1252')
    temp[col]=uppper(temp,col)
    model_temp = pd.DataFrame()
    labels_temp_map = dict(zip(temp[col],temp['Id (Encoding)']))
    model_temp[col_1] = model[col_1].map(labels_temp_map)
    print(model.head())
    return model_temp[col_1]

In [68]:
def df_reset(df):
    return pd.DataFrame()

In [69]:
data.rename(columns={'Make':'Make_Clean'},inplace=True)
data.rename(columns={'Model':'Model_Clean'},inplace=True)
data.rename(columns={'Variant':'Variant_Clean'},inplace=True)
data.rename(columns={'GVW_HP_SEATING_CAPACITY_CC':'num_Weight'},inplace=True)


In [70]:
data.columns

Index(['UNIQUESERIALNO', 'SOLD_DATE', 'MAKE YEAR', 'Make_Clean', 'Model_Clean',
       'Variant_Clean', 'FUEL TYPE_CLEANED', 'mixmmvfuelcleaned', 'BodyType',
       'SUBBODYTYPE', 'WHEELS', 'num_Weight', 'SOLD AMOUNT', 'STATE_MAPPED',
       'VEHICLE NO', 'METER READING', 'UCR', 'SHROTCOND_MAPPED', 'INSDT',
       'TAX', 'PERMIT_MAPPED', 'SELLERCORPORATENAME', 'SELLERID', 'SELLERNAME',
       'KEYREFNUMBER', 'INV_H_RC_STATUS_CLEANED_MAPPED', 'VEHICLECONDITION',
       'SELLER SEGMENT'],
      dtype='object')

In [71]:
data.rename(columns={'FUEL TYPE_CLEANED':'Fuel_Clean'},inplace=True)
data.rename(columns={'STATE_MAPPED':'CV_State_Clean'},inplace=True)
data.rename(columns={'MAKE YEAR':'MAKE_YEAR'},inplace=True)

In [72]:
data['Make_Clean']=uppper(data,'Make_Clean')
data['Model_Clean']=uppper(data,'Model_Clean')
data['Variant_Clean']=uppper(data,'Variant_Clean')
data['Fuel_Clean']=uppper(data,'Fuel_Clean')
data['CV_State_Clean']=uppper(data,'CV_State_Clean')

In [73]:
dt = data.dropna(how='any',axis=0,subset=['Model_Clean','Fuel_Clean','Variant_Clean'])

In [74]:
dt['MMV']= dt['Make_Clean']+'_'+dt['Model_Clean']+'_'+dt['Variant_Clean']

In [75]:
df_1= dt.copy()

In [76]:
df_1.shape

(401773, 29)

In [77]:
df_2 = df_1[df_1['SOLD AMOUNT']>48000] 

In [78]:
df_2.shape

(364151, 29)

In [79]:
df_3 = df_2[df_2['MMV'].map(df_2['MMV'].value_counts()) >=10]

In [80]:
df_3.shape

(363909, 29)

In [81]:
df_1 =df_reset(df_1)

In [82]:
df_2 = df_reset(df_2)

In [83]:
df = df_3.copy()

In [84]:
df_1 = df[df['MAKE_YEAR']>=1995]

In [85]:
df_1.shape

(361651, 29)

In [86]:
df_1.CV_State_Clean.fillna(df_1.CV_State_Clean.mode()[0],inplace=True)

In [87]:
df_1.SOLD_DATE=pd.to_datetime(df_1.SOLD_DATE)

In [88]:
df_1["Year_sell"] = df_1.SOLD_DATE.dt.year

In [89]:
df_1["Sell_Month"] = df_1.SOLD_DATE.dt.month

In [90]:
df_1["Sell_Day"] = df_1.SOLD_DATE.dt.day

In [91]:
#df_2 = df_1[df_1["Year_sell"]]

In [92]:
df_1["num_CV_Age"] = df_1["Year_sell"] - df_1['MAKE_YEAR']

In [93]:
df_2 = df_1[df_1["num_CV_Age"]>0]

In [94]:
df_2.shape

(343864, 33)

In [95]:
#df_1[df_1["BodyType"]=='TRACTOR TRAILER' ].value_counts

df_2.head()

,UNIQUESERIALNO,SOLD_DATE,MAKE_YEAR,Make_Clean,Model_Clean,Variant_Clean,Fuel_Clean,mixmmvfuelcleaned,BodyType,SUBBODYTYPE,WHEELS,num_Weight,SOLD AMOUNT,CV_State_Clean,VEHICLE NO,METER READING,UCR,SHROTCOND_MAPPED,INSDT,TAX,PERMIT_MAPPED,SELLERCORPORATENAME,SELLERID,SELLERNAME,KEYREFNUMBER,INV_H_RC_STATUS_CLEANED_MAPPED,VEHICLECONDITION,SELLER SEGMENT,MMV,Year_sell,Sell_Month,Sell_Day,num_CV_Age
0,S20091800155,2020-11-26,2013,AMW,2516HL,6X2CARGOTRUCK,DIESEL,AMW2516HL6X2CARGOTRUCKDIESEL,HAULAGE TRUCK,10 WHEELS / GVW 25000KG TO 28500KG /RIGID BODY,10,25000,445000,GUJARAT,GJ16Z6727,490620.000,20%,FAIR,Live : 28/02/2021,Expired : 30/09/2019,NOT AVAILABLE,a9,AP000337591,JAYESHBHAI ZAVERBHAI PATEL,BHARUO608090001,ORIGINAL RC,MOVABLE,RETAIL,AMW_2516HL_6X2CARGOTRUCK,2020,11,26,7
1,S21092900303,2021-09-11,2010,AMW,2516HL,6X2CARGOTRUCK,DIESEL,AMW2516HL6X2CARGOTRUCKDIESEL,HAULAGE TRUCK,10 WHEELS / GVW 25000KG TO 28500KG /RIGID BODY,10,25000,315000,TAMILNADU,TN22BT4664,736100.000,0.2,FAIR,Live : 29/03/2022,Expired : 31/03/2021,NOT AVAILABLE,a9,AC000156811,CHINNAVELLAI CHINNAVELLAI,ARUPKT908140010,Original RC,IMMOVABLE,Insurance,AMW_2516HL_6X2CARGOTRUCK,2021,9,11,11
2,MKTCV1,2022-01-01,2005,AMW,2516HL,6X2CARGOTRUCK,DIESEL,AMW2516HL6X2CARGOTRUCKDIESEL,HAULAGE TRUCK,10 WHEELS / GVW 25000KG TO 28500KG /RIGID BODY,10,25000,231646,MADHYAPRADESH,NaN,1112310.000,40-50%,GOOD,LIVE,VALID,VALID,NaN,NaN,NaN,NaN,ORIGINAL RC,MOVABLE,OEM,AMW_2516HL_6X2CARGOTRUCK,2022,1,1,17
3,MKTCV2,2021-01-01,2005,AMW,2516HL,6X2CARGOTRUCK,DIESEL,AMW2516HL6X2CARGOTRUCKDIESEL,HAULAGE TRUCK,10 WHEELS / GVW 25000KG TO 28500KG /RIGID BODY,10,25000,243837,BIHAR,NaN,1050260.000,40-50%,GOOD,LIVE,VALID,VALID,NaN,NaN,NaN,NaN,ORIGINAL RC,MOVABLE,OEM,AMW_2516HL_6X2CARGOTRUCK,2021,1,1,16
4,MKTCV3,2022-01-01,2006,AMW,2516HL,6X2CARGOTRUCK,DIESEL,AMW2516HL6X2CARGOTRUCKDIESEL,HAULAGE TRUCK,10 WHEELS / GVW 25000KG TO 28500KG /RIGID BODY,10,25000,243837,HARYANA,NaN,1050260.000,40-50%,GOOD,LIVE,VALID,VALID,NaN,NaN,NaN,NaN,ORIGINAL RC,MOVABLE,OEM,AMW_2516HL_6X2CARGOTRUCK,2022,1,1,16


In [96]:
df_2[['TAX_STATUS','TAX_DATE']] = df_2['TAX'].str.split(":",expand=True)

df_2['TAX_STATUS']= df_2['TAX_STATUS'].str.lstrip()
df_2['TAX_STATUS']= df_2['TAX_STATUS'].str.rstrip()

df_2['TAX_STATUS']=df_2['TAX_STATUS'].str.upper()

df_2['TAX_STATUS'].value_counts()

TAX_STATUS={
    "LIFE TIME":"LIFE_TIME",
    "NOT AVAILABLE":"NOT_AVAILABLE",
    "PAID UPTO":"PAID_UPTO",
    "EXPIRED":"EXPIRED",
    "VALID UPTO":"VALID_UPTO",
    "NOT APPLICABLE":"NOT_APPLICABLE",
    "LIFE TAX":"LIFE_TAX",
    "DATE IS NOT AVAILABLE":"DATE_NOT_AVAILABLE"
    }

df_2['TAX_STATUS'].replace(TAX_STATUS, inplace=True)

In [97]:
df_2['TAX_STATUS'].value_counts()

VALID                 93128
LIFE_TIME             78219
PAID_UPTO             61708
NOT_AVAILABLE         50203
EXPIRED               46331
VALID_UPTO             6408
LIFETIME               2138
NOT_APPLICABLE         1602
PAIDUPTO               1583
LIFE_TAX               1494
NOTAVAILABLE            430
VALIDUPTO               318
DATE_NOT_AVAILABLE      105
LIFETAX                  91
NOTAPPLICABLE            34
Name: TAX_STATUS, dtype: int64

In [98]:
df_2.columns

Index(['UNIQUESERIALNO', 'SOLD_DATE', 'MAKE_YEAR', 'Make_Clean', 'Model_Clean',
       'Variant_Clean', 'Fuel_Clean', 'mixmmvfuelcleaned', 'BodyType',
       'SUBBODYTYPE', 'WHEELS', 'num_Weight', 'SOLD AMOUNT', 'CV_State_Clean',
       'VEHICLE NO', 'METER READING', 'UCR', 'SHROTCOND_MAPPED', 'INSDT',
       'TAX', 'PERMIT_MAPPED', 'SELLERCORPORATENAME', 'SELLERID', 'SELLERNAME',
       'KEYREFNUMBER', 'INV_H_RC_STATUS_CLEANED_MAPPED', 'VEHICLECONDITION',
       'SELLER SEGMENT', 'MMV', 'Year_sell', 'Sell_Month', 'Sell_Day',
       'num_CV_Age', 'TAX_STATUS', 'TAX_DATE'],
      dtype='object')

In [99]:
df_2['METER_READING_cleaned'] = df_2['METER READING'].str.replace(r'[^\d.]+""', '')

AttributeError: Can only use .str accessor with string values!

In [ ]:
df_2['METER_READING_cleaned']=pd.to_numeric(df_2['METER_READING_cleaned'],errors='coerce')

In [ ]:
df_2['METER_READING_cleaned'].describe()

In [ ]:
meter_reading_upper=df_2['METER_READING_cleaned'].mean()+3*df_2['METER_READING_cleaned'].std()
meter_reading_upper

In [ ]:
df_2=df_2[~(df_2['METER_READING_cleaned']>meter_reading_upper)]

In [ ]:
df_2.shape

In [ ]:
df_2['METER_READING_cleaned'].describe()

In [ ]:
df_21=df_2.copy()

In [ ]:
print(df_2[df_2.METER_READING_cleaned == df_2.METER_READING_cleaned.max()])  



In [ ]:
df_2['METER_READING_cleaned'] = df_2['METER_READING_cleaned'].replace(0, np.nan)

In [ ]:
#df_2['METER_READING_cleaned'] = df_2['METER_READING_cleaned'].replace([9281120,9231261,9111099,9006008,8817191,8164387,8040906,8038913,7774861,7748961,7674216,7627843], np.nan)

In [ ]:
df_2['METER_READING_cleaned'].values[df_2['METER_READING_cleaned'] > 4000000] = 141557

In [ ]:
df_2['METER_READING_cleaned'].describe()

In [ ]:
df_2['WHEELS'].describe()

In [ ]:
df_2['WHEELS'] = df_2['WHEELS'].replace({0:np.nan})

In [ ]:
df_2['num_Weight'].describe()

In [ ]:
df_2['num_Weight'] = df_2['num_Weight'].replace({0:np.nan})

In [ ]:
df_2['num_Weight'].describe()

In [ ]:
df_2['MMV']=df_2['Make_Clean']+df_2['Model_Clean']+df_2['Variant_Clean']

In [ ]:
df_2.isnull().sum()/len(df_2)

In [ ]:
pct_null = df_2.isnull().sum() / len(df_2)
missing_features = pct_null[pct_null > 0.70].index
# df.drop(missing_features, axis=1, inplace=True)

In [ ]:
missing_features

In [ ]:
inp1=df_2.select_dtypes(exclude='object')

In [ ]:
inp1.shape

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (8,8))        # Size of the figure
sns.heatmap(inp1.corr(),annot = True)
plt.show()

In [ ]:
Unseen_df = df_2.loc[(df_2['SOLD_DATE'] >= '2022-03-01')]

In [ ]:
Unseen_for_model=Unseen_df[['MMV','Fuel_Clean','BodyType', 'MAKE_YEAR','Make_Clean','Model_Clean','Variant_Clean',
       'SOLD AMOUNT', 'CV_State_Clean','SHROT COND_MAPPED','num_Weight','Year_sell',
       'num_CV_Age','PERMIT_MAPPED','INV_H_CATEGORY_mapping','METER_READING_cleaned',
           'INV_H_RC_STATUS_CLEANED_MAPPED','GP_PURPOSE_CLEANED','VEHICLECONDITION']]

In [ ]:
mydata=df_2.loc[(df_2['SOLD_DATE'] < '2022-03-01')]

In [ ]:
df_2=mydata[['MMV','Fuel_Clean','BodyType', 'MAKE_YEAR','Make_Clean','Model_Clean','Variant_Clean',
       'SOLD AMOUNT', 'CV_State_Clean','SHROT COND_MAPPED','num_Weight','Year_sell',
       'num_CV_Age','PERMIT_MAPPED','INV_H_CATEGORY_mapping','METER_READING_cleaned',
           'INV_H_RC_STATUS_CLEANED_MAPPED','GP_PURPOSE_CLEANED','VEHICLECONDITION']]

In [ ]:
df_2.shape

In [ ]:
columns=['MMV','Fuel_Clean','BodyType', 'MAKE_YEAR','Make_Clean','Model_Clean','Variant_Clean',
       'SOLD AMOUNT', 'CV_State_Clean','SHROT COND_MAPPED','num_Weight','Year_sell',
       'num_CV_Age','PERMIT_MAPPED','INV_H_CATEGORY_mapping','METER_READING_cleaned',
           'INV_H_RC_STATUS_CLEANED_MAPPED','GP_PURPOSE_CLEANED','VEHICLECONDITION']

In [ ]:
#Feature Enginerring
df_2.reset_index(drop=True,inplace =True)

In [ ]:
df_2.columns

In [ ]:
Unseen_for_model.reset_index(drop=True,inplace =True)

In [ ]:
Model_data = df_2.copy()

In [ ]:
Model_data.shape

In [ ]:
cat_f=Model_data.select_dtypes(include='object').columns

In [ ]:
cat_f=['MMV', 'Fuel_Clean', 'BodyType',
       'CV_State_Clean','SHROT COND_MAPPED', 
       'PERMIT_MAPPED', 'INV_H_CATEGORY_mapping',
       'INV_H_RC_STATUS_CLEANED_MAPPED', 'GP_PURPOSE_CLEANED',
       'VEHICLECONDITION']

In [ ]:
num_f=Model_data.select_dtypes(exclude='object').columns

In [ ]:
num_f

In [ ]:
inp1=Model_data.select_dtypes(exclude='object')

In [ ]:
from scipy.stats import gaussian_kde
fig = plt.figure(figsize=(18,5))


for i,b in enumerate(list(inp1.columns)):
    
    i +=1
    ax = fig.add_subplot(1,6,i)
    sns.boxplot(inp1[b])

    ax.set_title(b)

sns.set_style("whitegrid")
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
columns=Model_data.select_dtypes(exclude='object').columns

In [ ]:
plt.subplots(figsize=(15,7))
 
for i, col in enumerate(inp1):
  plt.subplot(2,3,i+1)
  sns.distplot(inp1[col])
plt.show()

In [ ]:
columns

In [ ]:
inp_cat=Model_data.select_dtypes(include='object')

In [ ]:
inp_cat.columns

In [ ]:
Model_data.loc[Model_data['MMV'].isin((Model_data['MMV'].value_counts()[Model_data['MMV'].value_counts(normalize=True) < 0.005]).index), 'MMV'] = 'other'

In [ ]:
Model_data["PERMIT_MAPPED"].value_counts()

In [ ]:
Model_data['SHROT COND_MAPPED'].value_counts(normalize=True)

In [ ]:
Model_data.loc[Model_data['SHROT COND_MAPPED'].isin((Model_data['SHROT COND_MAPPED'].value_counts()[Model_data['SHROT COND_MAPPED'].value_counts(normalize=True) < 0.015]).index), 'SHROT COND_MAPPED'] = 'other'

In [ ]:
Model_data['SHROT COND_MAPPED'].value_counts(normalize=True)

In [ ]:
Model_data['PERMIT_MAPPED'].value_counts()

In [ ]:
Model_data['BodyType'].value_counts(normalize=True)

In [ ]:
Model_data.loc[Model_data['BodyType'].isin((Model_data['BodyType'].value_counts()[Model_data['BodyType'].value_counts(normalize=True) <= 0.020]).index), 'BodyType'] = 'other'

In [ ]:
Model_data['BodyType'].value_counts(normalize=True)

In [ ]:
Model_data['INV_H_RC_STATUS_CLEANED_MAPPED'].value_counts(normalize=True)

In [ ]:
Model_data.loc[Model_data['INV_H_RC_STATUS_CLEANED_MAPPED'].isin((Model_data['INV_H_RC_STATUS_CLEANED_MAPPED'].value_counts()[Model_data['INV_H_RC_STATUS_CLEANED_MAPPED'].value_counts(normalize=True) <= 0.013]).index), 'INV_H_RC_STATUS_CLEANED_MAPPED'] = 'other'

In [ ]:
Model_data['INV_H_RC_STATUS_CLEANED_MAPPED'].value_counts(normalize=True)

In [ ]:
Model_data['CV_State_Clean'].value_counts(normalize=True)

In [ ]:
Model_data['CV_State_Clean'].value_counts(normalize=True)

In [ ]:
Model_data.loc[Model_data['CV_State_Clean'].isin((Model_data['CV_State_Clean'].value_counts()[Model_data['CV_State_Clean'].value_counts(normalize=True) <= 0.009]).index), 'CV_State_Clean'] = 'other'

In [ ]:
Model_data.rename(columns={'CV_Age':'num_CV_Age'},inplace=True)
Model_data.rename(columns={'METER_READING_cleaned':'num_METER_READING_cleaned'},inplace=True)
Model_data.rename(columns={'SOLD AMOUNT':'num_SOLD_AMOUNT'},inplace=True)

In [ ]:
Model_data.columns

In [ ]:
num_f=['WHEELS', 'num_Weight', 'num_SOLD AMOUNT', 'num_CV_Age','num_METER_READING_cleaned']

In [ ]:
num_features =  [col for col in Model_data if col.startswith('num_')]

In [ ]:
#from scipy.stats import gaussian_kde
#fig = plt.figure(figsize=(18,5))
#
#
#for i,b in enumerate(list(Model_data[num_features].columns)):
#    
#    i +=1
#    ax = fig.add_subplot(1,5,i)
#    sns.boxplot(Model_data[num_features][b])
#
#    ax.set_title(b)
#
#sns.set_style("whitegrid")
#plt.tight_layout()
#plt.legend()
#plt.show()

In [ ]:
#def remove_outlier_IQR(df):
#    Q1=df.quantile(0.05)
#    Q3=df.quantile(0.95)
#    IQR=Q3-Q1
#    df_final=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
#    return df_final

In [ ]:
#Model_data[num_features] = remove_outlier_IQR(Model_data[num_features])

In [ ]:
#finding the 0th moment 
print('0th moment is =',stats.moment(Model_data[num_features],moment=0))
#finding the 1st moment 
print('1st moment is =',stats.moment(Model_data[num_features],moment=1))
#finding the 2nd moment 
print('2nd moment is =',stats.moment(Model_data[num_features],moment=2))
#finding the 3rd moment 
print('3rd moment is =',stats.moment(Model_data[num_features],moment=3))
#finding the 4th moment 
print('4th moment is =',stats.moment(Model_data[num_features],moment=4))

In [ ]:
#from scipy.stats import gaussian_kde
#fig = plt.figure(figsize=(18,5))
#
#
#for i,b in enumerate(list(Model_data[num_features].columns)):
#    
#    i +=1
#    ax = fig.add_subplot(1,5,i)
#    sns.boxplot(Model_data[num_features][b])
#
#    ax.set_title(b)
#
#sns.set_style("whitegrid")
#plt.tight_layout()
#plt.legend()
#plt.show()


In [ ]:
Model_data.isnull().sum()/len(Model_data)

In [ ]:
#!pip install verstack

In [ ]:
#from verstack import NaNImputer


In [ ]:
#imputer = NaNImputer(fill_nans_in_pure_text = True)
#Model_data = imputer.impute(Model_data)#

In [ ]:
print(Model_data.MMV.value_counts(),'\n')

In [ ]:
#$ pip install verstack

In [ ]:
Model_data.fillna(Model_data.select_dtypes(include='number').median().iloc[0], inplace=True)

In [ ]:
Model_data.fillna(Model_data.select_dtypes(include='object').mode().iloc[0], inplace=True)

In [ ]:
#Model_data.to_csv('Model_dataxx.csv')

In [ ]:
Model_data_moment1=Model_data.groupby('MMV', as_index=False)['num_SOLD_AMOUNT']

In [ ]:
Model_data_moment1=Model_data.groupby('MMV', as_index=False)['num_SOLD_AMOUNT'].mean().rename(columns = {'num_SOLD_AMOUNT':'Moment_1'})
#Model_data_moment1

In [ ]:

Model_data_m1=Model_data.merge(Model_data_moment1,on='MMV',how='left')
#Model_data_m1.head()

In [ ]:
Model_data_moment2=Model_data_m1.groupby('MMV', as_index=False)['num_SOLD_AMOUNT'].var().rename(columns = {'num_SOLD_AMOUNT':'Moment_2'})
#Model_data_moment2

In [ ]:
Model_data_m2=Model_data_m1.merge(Model_data_moment2,on='MMV',how='left')

In [ ]:
Model_data_moment3=Model_data_m2.groupby('MMV', as_index=False)['num_SOLD_AMOUNT'].skew(axis = 0, skipna = True).rename(columns = {'num_SOLD_AMOUNT':'Moment_3'})


In [ ]:
#Model_data_m2.head()

In [ ]:
Model_data_m3=Model_data_m2.merge(Model_data_moment3,on='MMV',how='left')

In [ ]:
#Model_data_m3

In [ ]:
Model_data_moment4=Model_data_m3.groupby('MMV', as_index=False)['num_SOLD_AMOUNT'].apply(pd.DataFrame.kurt).rename(columns = {'num_SOLD_AMOUNT':'Moment_4'})

In [ ]:
#Model_data_moment4

In [ ]:
Model_moment=Model_data_m3.merge(Model_data_moment4,on='MMV',how='left')

In [ ]:
Model_data=Model_moment.copy()
Model_data

In [ ]:
Model_data
df_inf

In [ ]:
merged_df = pd.merge(Model_data, df_inf, on='Year_sell')

In [ ]:
merged_df.head()

In [ ]:
#merged_df['Inf_Pric'] = merged_df.apply(lambda row: row['num_SOLD_AMOUNT'] * (1 + 'Inf_rate'), axis=1)


In [ ]:
#df['Inf_rate'] = df['Inf_rate'].str.rstrip('%').astype(float) / 100

In [ ]:
Model_data.isnull().sum()

In [ ]:
Model_data['Year_sell'].agg(['min', 'max'])


In [ ]:
Model_data['MAKE_YEAR'].agg(['min', 'max'])

In [ ]:
#Model_data['Inf_Pric'] = Model_data['num_SOLD_AMOUNT'] * df_inf.set_index['Inf_rate']
# define a lambda function to multiply inflated price and sold amount
#_inflated_sold = lambda row: row['Inf_rate'] * row['num_SOLD_AMOUNT']



In [ ]:
# apply the lambda function to each row to calculate inflated sold amount
#merged_df['Inf_Pric'] = merged_df.apply(_inflated_sold, axis=1)

In [ ]:
#merged_df.head()

In [ ]:
#_inflated_sold = lambda row: row['Inf_Pric'] + row['num_SOLD_AMOUNT']

In [ ]:
#merged_df['Inf_Pric@'] = merged_df.apply(_inflated_sold, axis=1)

In [ ]:
#merged_df.head()

In [ ]:
Model_data.head()

In [ ]:
#Model_data_moment4=Model_data.groupby('MMV', as_index=False)['num_SOLD_AMOUNT']
#profile = pp.ProfileReport(Model_data)
#profile.to_file("output.html")

In [ ]:
#from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
#for i in range(len(Model_data.columns[:-1])):
#    v=vif(np.matrix(Model_data[:-1]),i)
#    print('Varince {}:{}'.format(Model_data[i],round(v,2)))

In [ ]:
#Model_data.merge(Model_data_moment1,on='MMV',how='left')
#Model_data.merge(Model_data_moment2,on='MMV',how='left')
#Model_data.merge(Model_data_moment3,on='MMV',how='left')
#Model_data.merge(Model_data_moment4,on='MMV',how='left')
#Model_data.head()

In [ ]:
one_hot_encoded_data = pd.get_dummies(Model_data, columns = ['MMV','Fuel_Clean', 'BodyType','MAKE_YEAR', 'Make_Clean', 'Model_Clean', 'Variant_Clean',
       'CV_State_Clean','SHROT COND_MAPPED', 
       'PERMIT_MAPPED', 'INV_H_CATEGORY_mapping',
       'INV_H_RC_STATUS_CLEANED_MAPPED', 'GP_PURPOSE_CLEANED',
       'VEHICLECONDITION'])

In [ ]:
one_hot_encoded_data.shape

In [ ]:
Model_data=one_hot_encoded_data.copy()

In [ ]:
Model_data.head()

In [ ]:
num_features=['num_Weight', 'num_CV_Age',
       'num_METER_READING_cleaned']

In [ ]:
#Q1 = num_f.quantile(0.25)
#Q3 = num_f.quantile(0.75)
#IQR = Q3 - Q1
#print(IQR)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaler = StandardScaler()
Model_data[num_features]=scaler.fit_transform(Model_data[num_features])

In [ ]:
Model_data.head()

In [ ]:
Model_data.info()

In [ ]:
#Model_data.drop(['Annual'],axis=1)

In [ ]:
#Model_data1=Model_data.copy()

In [ ]:
#from scipy import stats
#import numpy as np
#z = np.abs(stats.zscore(model_data1))
#print(z)

In [ ]:
#threshold = 3
#print(np.where(z > 3))

In [ ]:
#Q1 = model_data1.quantile(0.25)
#Q3 = model_data1.quantile(0.75)
#IQR = Q3 - Q1
#print(IQR)

In [ ]:
#print(model_data1 < (Q1 - 1.5 * IQR)) |(model_data1 > (Q3 + 1.5 * IQR))

In [ ]:
#model_data.info()
#Model_data=model_data.reset_index()

In [ ]:
X=Model_data.drop(['num_SOLD_AMOUNT'],axis=1)
y=Model_data[['num_SOLD_AMOUNT']]

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=0.25,
                                                                    shuffle=True,
                                                                    random_state=0)
print(y.shape[0])
print(y_train.shape[0])
print(y_test.shape[0])

In [ ]:
!pip install xgboost


In [ ]:
import xgboost
regressor=xgboost.XGBRegressor()

In [ ]:
clf = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.1, gamma=0, learning_rate=0.005, max_delta_step=0.7,
       max_depth=10, min_child_num_Weight=2, missing=1, n_estimators=850,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=707,
       reg_alpha=10, reg_lambda=300, scale_pos_num_Weight=0.1, seed=42,
       silent=True, subsample=0.1)

In [ ]:
#clf = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
#       colsample_bytree=0.1, gamma=0, learning_rate=0.005, max_delta_step=0.7,
#       max_depth=10, min_child_num_Weight=2, missing=1, n_estimators=850,
#       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=707,
#       reg_alpha=10, reg_lambda=300, scale_pos_num_Weight=0.1, seed=42,
#       silent=True, subsample=0.1)

In [ ]:
clf.fit(X_train, y_train, early_stopping_rounds=900, eval_metric=["mape",'error','logloss'],
        eval_set=[(X_train, y_train), (X_test, y_test)],verbose=0)

In [ ]:
#import graphviz

In [ ]:
#clf.fit(X_train, y_train)
results = clf.evals_result()

plt.figure(figsize=(8,5))
plt.plot(results["validation_0"]["mape"], label="Training loss")
plt.plot(results["validation_1"]["mape"], label="Test loss")
plt.axvline(21, color="gray", label="Base")
plt.xlabel("Number of rounds")
plt.ylabel("Loss")
plt.legend()



In [ ]:
clf.best_ntree_limit

In [ ]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [ ]:
n_estimators = [100, 500, 900, 1100, 1500]
#n_estimators = [100, 500]
max_depth = [2, 3, 5, 8, 10, 12, 15]
learning_rate=[0.001,0.01,0.05,0.1,0.15,0.20,0.30]
min_child_num_Weight=[1,2,3,4,5,7]
#'min_child_weight': np.arange(0.1, 10.1, 0.1)
#gamma=[7,10]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_num_Weight':min_child_num_Weight,
    #'min_child_weight': np.arange(0.1, 10.1, 0.1),
    'booster':booster,
    'base_score':base_score,
    #'gamma':gamma
    }#

In [ ]:
# Set up the random search with 2-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=2,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42,
            refit=True)

In [ ]:
#i=1
#kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
#for train_index,test_index in kf.split(X,y):
#     print('\n{} of kfold {}'.format(i,kf.n_splits))
#     xtr,xvl = X.loc[train_index],X.loc[test_index]
#     ytr,yvl = y[train_index],y[test_index]
#     model = GridSearchCV(XGBClassifier() ,param_distributions=hyperparameter_grid, cv=10, scoring= 'f1',iid=True)
#     model.fit(xtr, ytr)
#     print (model.best_params_)
#     pred=model.predict(xvl)
#     print('accuracy_score',accuracy_score(yvl,pred))
#     i+=1

In [ ]:
random_cv.fit(X_train,y_train)


In [ ]:
 random_cv.best_params_, random_cv.best_score_, random_cv.best_estimator_

In [ ]:
y_train_pred=random_cv.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_train,y_train_pred)

In [ ]:
X_train_copy=X_train.copy()

In [ ]:
X_train_copy['Predicted']=y_train_pred

In [ ]:
X_train_copy['Actual']=y_train

In [ ]:
from xgboost import plot_tree
import matplotlib.pyplot as plt



In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz\bin.dot'

In [ ]:
incorrect = X_test_copy[X_test_copy["Actual"] != X_test_copy["Predicted"]]

In [ ]:
incorrect

In [ ]:
X_test_copy['Abs Error'] = abs(X_test_copy['Actual'] - X_test_copy['Predicted'])

# Calculate the percentile rank of the absolute error within the test data set
X_test_copy['Error Rank'] = X_test_copy['Abs Error'].rank(pct=True)

# Select the top 5% of highly wrong predictions
highly_wrong_predictions = X_test_copy[X_test_copy['Error Rank'] >= 0.95]

# Print the highly wrong predictions
#print(highly_wrong_predictions)
highly_wrong_predictions.head(20)

In [ ]:
highly_wrong_predictions1 = highly_wrong_predictions[highly_wrong_predictions['MMV']=='other']

In [ ]:
X_train_copy['Predicted']

In [ ]:
X_train_copy.head()

In [ ]:
MAPE=abs((X_train_copy['Actual']-X_train_copy['Predicted'])/X_train_copy['Actual']).mean()*100

In [ ]:
MAPE

In [ ]:
# Plot the histogram of the error terms
import matplotlib.pyplot as plt
import seaborn as sns
fig = plt.figure()
sns.distplot((X_train_copy['Actual'] - X_train_copy['Predicted']), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  
plt.xlabel('Errors', fontsize = 18)  
plt.show()

In [ ]:
y_test_pred=random_cv.predict(X_test)

In [ ]:
r2_score(y_test,y_test_pred)

In [ ]:
MAPE_test=abs((X_test_copy['Actual']-X_test_copy['Predicted'])/X_test_copy['Actual']).mean()*100

In [ ]:
MAPE_test

In [ ]:
plt.ion()
plt.figure(figsize=(10,10))
plt.scatter(X_train_copy['Actual'], X_train_copy['Predicted'], c='crimson')
#plt.yscale('log')
#plt.xscale('log')

p1 = max(max(X_train_copy['Predicted']), max(X_train_copy['Actual']))
p2 = min(min(X_train_copy['Predicted']), min(X_train_copy['Actual']))
plt.plot([p1, p2], [p1, p2])
plt.xlabel('Actual', fontsize=15)
plt.ylabel('Predicted', fontsize=15)
plt.axis('equal')
plt.ion()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.set_context("notebook")
ax1 = sns.distplot(X_train_copy['Actual'], hist=False, color="r", label="Actual Value")
sns.distplot(X_train_copy['Predicted'], hist=False, color="b", label="Fitted Values" , ax=ax1)
plt.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
p1 = max(max(X_train_copy['Predicted']), max(X_train_copy['Actual']))
p2 = min(min(X_train_copy['Predicted']), min(X_train_copy['Actual']))
#px.plot([p1, p2], [p1, p2])
#px.xlabel('Actual', fontsize=15)
#px.ylabel('Predicted', fontsize=15)
#px.axis('equal')
#px.show()
#df = px.data.tips()
fig = px.scatter(X_train_copy, x = "Actual", y = "Predicted")
fig.add_traces(go.Scatter(x=[p1,p2], y=[p1,p2], name='Regression Fit'))
fig.show()

In [ ]:
y_test_pred=random_cv.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:

predictions = [round(value) for value in y_test_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = clf.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()
# plot classification error
#fig, ax = plt.subplots()
#ax.plot(x_axis, results['validation_0']['error'], label='Train')
#ax.plot(x_axis, results['validation_1']['error'], label='Test')
#ax.legend()
#plt.ylabel('Classification Error')
#plt.title('XGBoost Classification Error')
#plt.show()

In [ ]:
r2_score(y_test,y_test_pred)

In [ ]:
X_test_copy=X_test.copy()
X_test_copy['Predicted']=y_test_pred
X_test_copy['Actual']=y_test


In [ ]:
X_test_copy['Predicted']

In [ ]:
X_test_copy.head()

In [ ]:
#X_test = Model_data.transform(X_test)

In [ ]:
MAPE_test=abs((X_test_copy['Actual']-X_test_copy['Predicted'])/X_test_copy['Actual']).mean()*100

In [ ]:
MAPE_test

In [ ]:
plt.figure(figsize=(10,8))
sns.set_context("notebook")
ax1 = sns.distplot(X_test_copy['Actual'], hist=False, color="r", label="Actual Value")
sns.distplot(X_test_copy['Predicted'], hist=False, color="b", label="Fitted Values" , ax=ax1)
plt.show()

In [ ]:
# Plot the histogram of the error terms
import matplotlib.pyplot as plt
import seaborn as sns
fig = plt.figure()
sns.distplot((X_test_copy['Actual'] - X_test_copy['Predicted']), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  
plt.xlabel('Errors', fontsize = 18)  
plt.show()

In [ ]:
from xgboost import plot_tree
import graphviz

In [ ]:
# Fit the model
#_model_ = smf.ols("outcome ~ predictor", data=Model_data).fit()

# Train the XGBRegressor model
#model = xgb.XGBRegressor()
#model.fit(X_train, y_train)
#
# Plot the first tree of the model
#plt.figure(figsize=(20, 20))
#regressor.plot_tree(clf, num_trees=0)
#plt.show()


# Train the XGBRegressor model
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Plot the first tree of the model
dot_data = plot_tree(model, num_trees=0, rankdir='LR')
graph = graphviz.Source(dot_data)
graph.render('tree_plot')

In [ ]:
# Print the summary statistics
#print(_model_.summary())

# Make predictions
#predictions = model.predict(Model_data)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_test_copy['Actual'], X_test_copy['Predicted'], c='crimson')
#plt.yscale('log')
#plt.xscale('log')

p1 = max(max(X_test_copy['Predicted']), max(X_test_copy['Actual']))
p2 = min(min(X_test_copy['Predicted']), min(X_test_copy['Actual']))
plt.plot([p1, p2], [p1, p2])
plt.xlabel('Actual', fontsize=15)
plt.ylabel('Predicted', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    early_stopping_rounds = range(1, len(train_metrics) + 1)
    plt.plot(early_stopping_rounds, train_metrics)
    plt.plot(early_stopping_rounds, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_metric(history, 'loss')

In [ ]:

import statsmodels.formula.api as smf

# Load the data
#df = pd.read_csv("data.csv")

# Fit the model
model = smf.ols("outcome ~ predictor", data=Model_data).fit()

# Print the summary statistics
print(model.summary())

# Make predictions
predictions = model.predict(Model_data)






